In [1]:
import os, sys
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Loading Data

In [2]:
# Load data configuration
data_dir = r'C:\Users\alail\OneDrive\Documentos\dado_iniciais'
pen = [10, 50, 75]

df_cases = pd.read_csv(
    os.path.join(data_dir, 'cenarios_falta_preprocessamento.csv'),
    header=None,
    index_col=False
)

In [3]:
# Load data
if os.path.exists(os.path.join(data_dir, 'data.npy')):
    data = np.load(os.path.join(data_dir, 'data.npy'))
else:
    one_data_size = (6, 3072)
    data = np.zeros((len(pen) * len(df_cases), *one_data_size))
    for i, p in enumerate(pen):
        for j, case in df_cases.iterrows():
            data[i * len(df_cases) + j] = np.loadtxt(
                os.path.join(data_dir, f'pen{p}', f'sinal_{j+1}_.csv'),
                delimiter=','
            ).T
    np.save(os.path.join(data_dir, 'data.npy'), data) # save in numpy for faster loading

In [4]:
# Create a Dataframe to store the features
columns_name = [
    'File ID',
    'Fault type',
    'Fault resistance', 
    'Incidence angle',
    'Fault location',
    'Fault distance',
    'Pen',
]
columns_type = {
    'File ID': int,
    'Fault type': int,
    'Fault resistance': float,
    'Incidence angle': float,
    'Fault location': int,
    'Fault distance': float,
    'Pen': int
}
df_features = pd.DataFrame(
    columns= columns_name,
    index=range(len(pen) * len(df_cases))
)
for i, p in enumerate(pen):
    for j, case in df_cases.iterrows():
        df_features.loc[i * len(df_cases) + j] = case.tolist() + [p]
df_features = df_features.astype(columns_type)
del df_cases

### Fourier Features

In [5]:
from Features_Fourier import dft_features
idx_left = data.shape[-1]//2 - 256
idx_right = data.shape[-1]//2 + 512*2
df_fourier = df_features.copy()
fs = 512*60
for i in tqdm(range(data.shape[0])):
    feat = dft_features(data[i,::-1,idx_left:idx_right], fs)
    for j, f in enumerate(feat):
        df_fourier.loc[i, f'Feature {j}'] = f 
df_fourier.to_csv(r'features/fourier_features.csv', index=True)

  0%|          | 0/4050 [00:00<?, ?it/s]

### Wavelet Features

In [6]:
from MetricasWavelet import metricas_yordanos
df_wavelet = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = metricas_yordanos(data[i,3:6,:].T)
    for j, f in enumerate(feat):
        df_wavelet.loc[i, f'Feature {j}'] = f
df_wavelet.to_csv(r'features/wavelet_features.csv', index=True)

  0%|          | 0/4050 [00:00<?, ?it/s]

### Stockwell Features

In [7]:
from Features_Stockwell import extract_feature_TS
df_stockwell = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = np.concatenate([
        extract_feature_TS(data[i,3,:], tempo_inicio = data.shape[-1]/fs/2, sample_cycle=512),
        extract_feature_TS(data[i,4,:], tempo_inicio = data.shape[-1]/fs/2, sample_cycle=512),
        extract_feature_TS(data[i,5,:], tempo_inicio = data.shape[-1]/fs/2, sample_cycle=512)
    ])
    for j, f in enumerate(feat):
        df_stockwell.loc[i, f'Feature {j}'] = f
df_stockwell.to_csv(r'features/stockwell_features.csv', index=True)

  0%|          | 0/4050 [00:00<?, ?it/s]

### Three-Phase Features

In [10]:
from Features_TT import extract_feature_TT
df_tt = df_features.copy()
for i in tqdm(range(data.shape[0])):
    feat = extract_feature_TT(
        signal_voltage_A = data[i,0,:],
        signal_voltage_B = data[i,1,:],
        signal_voltage_C = data[i,2,:],
        signal_corrent_A = data[i,3,:],
        signal_corrent_B = data[i,4,:],
        signal_corrent_C = data[i,5,:],
        tempo_inicio = data.shape[-1]/fs/2,
        sample_cycle = 512
    )
    for j, f in enumerate(feat):
        df_tt.loc[i, f'Feature {j}'] = f
df_tt.to_csv(r'features/tt_features.csv', index=True) 

  0%|          | 0/4050 [00:00<?, ?it/s]